In [32]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import os
import librosa
from librosa.feature import spectral_centroid
# Mount Google Drive for access
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:

# Function to extract 5-second  chunks from audio
def extract_5sec_chunks(
    audio_array: np.ndarray,
    window_size_s: float = 8.0,
    hop_size_s: float = 4.0, #(with overlap)
    sample_rate=16000,
    # n_fft=2048,
    # n_mfcc=13,
) -> np.ndarray:
    frame_length = int(window_size_s * sample_rate)
    frame_step = int(hop_size_s * sample_rate)
    framed_audio = tf.signal.frame(audio_array, frame_length, frame_step, pad_end=False)

    return framed_audio



In [34]:
def spectral_centroid_feature(audio_array, sample_rate=22050, n_fft=2048):
    spec_centroid = spectral_centroid(y=audio_array, sr=sample_rate, n_fft=n_fft)
    return spec_centroid.T  # Transpose the array to have shape (n_samples, n_features)


In [35]:
def spectral_centroid_chunks(dataframe):
    y = []
    X = []

    for i, row in dataframe.iterrows():
        label = row['primary_label']
        tensor_5sec = row['audio_chunks']

        for each in tensor_5sec:
            y.append(label)
            each = np.array(each)
            X_spec_centroid = spectral_centroid_feature(each)  # Compute Spectral Centroid
            X_spec_centroid = X_spec_centroid.reshape(1, -1)  # Reshape to have a single sample dimension
            X.append(X_spec_centroid)

    assert len(y) == len(X)

    y = np.array(y)
    X = np.concatenate(X, axis=0)

    return X, y



In [36]:


# Load metadata
dataset_path = '/content/drive/MyDrive/207-Project/notebooks/RG/3_species/'
metadata_path = os.path.join(dataset_path, "train_val.csv")
metadata_df = pd.read_csv(metadata_path)

# Create a dictionary to map filenames to labels
labels_dict = metadata_df.set_index('filename_npy')['primary_label'].to_dict()

# # Get unique filenames from the metadata
# filenames = set(base_directory_path + '/' + metadata_df['filename_npy'].unique())

# Split train and validation data
train_df = metadata_df[metadata_df['data'] == 'train']
validate_df = metadata_df[metadata_df['data'] == 'val']

In [37]:
  # Load all train audio data one time
  train_audios = []
  for filename in train_df['filename_npy']:
      audio = np.load('/content/drive/MyDrive/207-Project/data/train/librosa_loaded/' + filename)
      train_audios.append(audio)

In [38]:
# Load all validate audio data one time
val_audios = []
for filename in validate_df['filename_npy']:
    audio = np.load('/content/drive/MyDrive/207-Project/data/train/librosa_loaded/' + filename)
    val_audios.append(audio)

In [39]:
print(len(train_audios))
print(len(val_audios))

657
283


In [40]:
#Initialize an empty list to store the chunk data for training data
train_chunks = []

for audio in train_audios:
  train_chunks.append(extract_5sec_chunks(audio))

len(train_chunks)

657

In [41]:
#Initialize an empty list to store the chunk data for VALIDATION data
val_chunks = []

for audio in val_audios:
  val_chunks.append(extract_5sec_chunks(audio))

len(val_chunks)

283

In [42]:
#add chunks to trainig data
train_df['audio_chunks'] = train_chunks
train_df.head()

train_df = train_df.sample(frac=1, random_state=1234)


<ipython-input-42-6ccab341e3d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['audio_chunks'] = train_chunks


In [43]:
#add chunks to validation data
validate_df['audio_chunks'] = val_chunks
validate_df.head()
validate_df = validate_df.sample(frac=1, random_state=1234)

<ipython-input-43-0cb3c619c1e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validate_df['audio_chunks'] = val_chunks


In [44]:
%who


LabelEncoder	 StandardScaler	 X_train	 X_train_list	 X_val	 X_val_list	 audio	 dataset_path	 drive	 
extract_5sec_chunks	 file	 filename	 gc	 json	 labels_dict	 librosa	 metadata_df	 metadata_path	 
np	 os	 pd	 spectral_centroid	 spectral_centroid_chunks	 spectral_centroid_feature	 tf	 train_audios	 train_chunks	 
train_df	 val_audios	 val_chunks	 validate_df	 y_train	 y_train_list	 y_val	 y_val_list	 


In [45]:
del audio
# del base_directory_path
del dataset_path
del drive
del extract_5sec_chunks
del filename
# del filenames
del labels_dict
# del layers
del metadata_df
del metadata_path
del os
del train_audios
del train_chunks
del val_audios
del val_chunks
# del train_test_split


In [46]:
import gc
gc.collect()

0

In [47]:
%who




LabelEncoder	 StandardScaler	 X_train	 X_train_list	 X_val	 X_val_list	 file	 gc	 json	 
librosa	 np	 pd	 spectral_centroid	 spectral_centroid_chunks	 spectral_centroid_feature	 tf	 train_df	 validate_df	 
y_train	 y_train_list	 y_val	 y_val_list	 


In [48]:
X_train, y_train = spectral_centroid_chunks(train_df)
X_val, y_val = spectral_centroid_chunks(validate_df)

In [49]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (4555, 251)
y_train shape: (4555,)
X_val shape: (1785, 251)
y_val shape: (1785,)


In [50]:
# validate_df.head()
print(train_df['primary_label'].unique())

['comsan' 'eaywag1' 'barswa']


In [51]:
del train_df
del validate_df
# del mfcc
# del mfcc_chunks
# del mfcc_to_audio

In [52]:
import json

In [53]:
X_train_list = X_train.tolist()
y_train_list = y_train.tolist()

X_val_list = X_val.tolist()
y_val_list = y_val.tolist()

# Save data to JSON files
with open('/content/drive/MyDrive/My-207/X_train_centroid.json', 'w') as file:
    json.dump(X_train_list, file)

with open('/content/drive/MyDrive/My-207/y_train_centroid.json', 'w') as file:
    json.dump(y_train_list, file)

with open('/content/drive/MyDrive/My-207/X_val_centroid.json', 'w') as file:
    json.dump(X_val_list, file)

with open('/content/drive/MyDrive/My-207/y_val_centroid.json', 'w') as file:
    json.dump(y_val_list, file)

In [54]:
# # Data normalization
# # Create the scaler
# scaler = MinMaxScaler()

# # Flatten X_train into a 2-dimensional array
# X_train = X_train.reshape(X_train.shape[0], -1)
# # Fit and transform the training data
# X_train = scaler.fit_transform(X_train)

# # Flatten X_val into a 2-dimensional array
# X_val = X_val.reshape(X_val.shape[0], -1)

# # Transform the validation data using the scaler parameters from the training data
# X_val = scaler.transform(X_val)

# # Convert string labels to numerical labels for training and validation data
# label_encoder = LabelEncoder()
# all_labels = np.array(list(labels_dict.values()))
# label_encoder.fit(all_labels)
# y_train_encoded = label_encoder.transform(y_train)
# y_val_encoded = label_encoder.transform(y_val)


In [55]:
# X_train_list = X_train.tolist()
# y_train_list = y_train.tolist()

# X_val_list = X_val.tolist()
# y_val_list = y_val.tolist()

# # Save data to JSON files
# with open('/content/drive/MyDrive/My-207/X_train_n.json', 'w') as file:
#     json.dump(X_train_list, file)

# with open('/content/drive/MyDrive/My-207/y_train_n.json', 'w') as file:
#     json.dump(y_train_list, file)

# with open('/content/drive/MyDrive/My-207/X_val_n.json', 'w') as file:
#     json.dump(X_val_list, file)

# with open('/content/drive/MyDrive/My-207/y_val_n.json', 'w') as file:
#     json.dump(y_val_list, file)

In [56]:
# # Data normalization
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_normalized = scaler.fit_transform(X_train)
# X_val_normalized = scaler.transform(X_val)

# # Convert string labels to numerical labels for training and validation data
# label_encoder = LabelEncoder()
# all_labels = np.array(list(labels_dict.values()))
# label_encoder.fit(all_labels)
# y_train_encoded = label_encoder.transform(y_train)
# # y_val_encoded = label_encoder.transform(y_val)

# # # Train the model
# # history = model.fit(X_train_normalized, y_train_encoded, validation_data=(X_val_normalized, y_val_encoded), batch_size=32, epochs=10)

# # Data normalization
# from sklearn.preprocessing import StandardScaler
# # Reshape X_train to 2D array
# num_samples, num_chunks, num_mfcc_features = X_train.shape
# X_train_reshaped = X_train.reshape(-1, num_mfcc_features)

# # Now apply the StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train_reshaped)

# # Reshape X_val to 2D array
# num_samples, num_chunks, num_mfcc_features = X_val.shape
# X_val_reshaped = X_val.reshape(-1, num_mfcc_features)

# # Now apply the StandardScaler using the same scaler fitted on X_train
# X_val = scaler.transform(X_val_reshaped)

# # Convert string labels to numerical labels for training and validation data
# label_encoder = LabelEncoder()
# all_labels = np.array(list(labels_dict.values()))
# label_encoder.fit(all_labels)
# y_train = label_encoder.transform(y_train)
# y_val = label_encoder.transform(y_val)

In [57]:
# print(labels_dict)

In [58]:

# # Build network topology
# model = tf.keras.Sequential([
#     # Input layer
#     tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),

#     # 1st dense layer
#     tf.keras.layers.Dense(512, activation='relu'),

#     # 2nd dense layer
#     tf.keras.layers.Dense(256, activation='relu'),

#     # 3rd dense layer
#     tf.keras.layers.Dense(64, activation='relu'),

#     # Output layer
#     tf.keras.layers.Dense(3, activation='softmax')
# ])


In [59]:

# # Compile model
# optimiser = tf.keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Summary of the model
# model.summary()


In [60]:

# # # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=10)

In [61]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (4555, 251)
y_train shape: (4555,)
X_val shape: (1785, 251)
y_val shape: (1785,)
